In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import xmltodict
import util
import math

from collections import OrderedDict

In [2]:
sheet = ET.parse('seed_template/seed_template.xml')

In [3]:
data = [
    (12, 48),
    (8, 32),
    (8, 32),
    (8, 32),
    (5, 32),
    (5, 32),
    (12, 48),
    (8, 21),
    (8, 21),
    (10, 21)
]

In [79]:
### SCHEMA
# <measure number="1">
#       <attributes>
#         <divisions>2</divisions>
#         </attributes>
#       <harmony print-frame="no">
#         <root>
#           <root-step>F</root-step>
#         </root>
#         <kind>major</kind>
#       </harmony>
#       <note>
#         <pitch>
#           <step>C</step>
#           <octave>4</octave>
#         </pitch>
#         <duration>2</duration>
#         <voice>1</voice>
#         <type>quarter</type>
#       </note>
# </measure>


# read XML
sheet = ET.parse('seed_template/seed_template.xml')

# STEP 1: find divisions
needed_divisions = [util.Constants.MIN_NEEDED_DIVISIONS[r[0]] for r in data]

def find_lcm(x):
    lcm = needed_divisions[0]
    for i in needed_divisions[1:]:
        lcm = lcm * i // math.gcd(lcm, i)
    return lcm

divs = find_lcm(needed_divisions)

# FILL MEASURES
part_xml = sheet.getroot().find('part')
measure_count = 0
for idx, d in enumerate(data):
    if d[0] == 12:
        if measure_count >= 1:
            part_xml.append(measure)
        
        measure = ET.Element('measure')
        measure_count += 1
        measure.attrib['number'] = str(measure_count)
        
        if measure_count == 1:
            attributes = ET.Element('attributes')
            divisions = ET.Element('divisions')
            divisions.text = str(divs)
            attributes.append(divisions)
            measure.append(attributes)
        
    else:
        root, mode = util.index_to_chord(d[1])
        duration = int(math.ceil(util.Constants.RHYTHMS[d[0]] * divs))
        
        harmony = ET.Element('harmony')
        harmony.attrib['print-frame'] = 'no'
        
        root_elem = ET.Element('root')
        rootstep_elem = ET.Element('root-step')
        rootstep_elem.text = root
        root_elem.append(rootstep_elem)
        harmony.append(root_elem)
        
        kind_elem = ET.Element('kind')
        kind_elem.text = mode.lower()
        harmony.append(kind_elem)
        
        measure.append(harmony)
        
        note = ET.Element('note')
        
        rest_elem = ET.Element('rest')
        rest_elem.text = ''
        note.append(rest_elem)
        
        dur_elem = ET.Element('duration')
        dur_elem.text = str(duration)
        note.append(dur_elem)
        
        voice_elem = ET.Element('voice')
        voice_elem.text = str(1)
        note.append(voice_elem)
        
        type_elem = ET.Element('type')
        type_elem.text = util.Constants.RHYTHM_TYPES[d[0]]
        note.append(type_elem)
        
        extra = util.Constants.RHYTHM_EXTRAS[d[0]]
        if extra:
            if extra == 'dot':
                dot_elem = ET.Element('dot')
                dot_elem.text = ''
                note.append(dot_elem)
            elif extra == 'time-modification':
                timemod_elem = ET.Element('time-modification')
                actual_elem = ET.Element('actual-notes')
                actual_elem.text = str(3)
                timemod_elem.append(actual_elem)
                
                normal_elem = ET.Element('normal-notes')
                normal_elem.text = str(2)
                timemod_elem.append(normal_elem)
                
                note.append(timemod_elem)
        
        measure.append(note)
        
if len(measure.getchildren()) > 0:
    part_xml.append(measure)
    
sheet.write('seed_template/seed_template_1.xml')

In [68]:
part_xml = sheet.getroot().find('part')
part_xml

<Element 'part' at 0x7fd80cf90b38>

In [26]:
sheet = ET.parse('seed_template/seed_1.xml')

In [49]:
sheet.getroot().find('part').getchildren()[0].getchildren()[0].getchildren()[0].text

'2'

In [42]:
measure = ET.Element('measure')
measure.getchildren()

[]

In [39]:
measure.attrib

{'number': '1'}

In [4]:
import mxl_generator

In [5]:
CRM = mxl_generator.ChordRhythmMXL('seed_template/seed_template.xml')

In [6]:
CRM.process(data)

In [7]:
CRM.write_mxl('xxxx.mxl')